# Metadata fra Geodata-info.dk

Denne notebook er tænkt til at hente data fra [geodata-info.dk](https://www.geodata-info.dk) (GDI) så egne geodata kan beriges udfra den nøgle (Identifikator) GDI tildeler deres datasæt. Det kræver således at man selv har tagget sine geodata med den nøglen og at disse kan udstilles, så data kan beriges med metadata. Vi arbejder med [Mapcentias GeoCloud2](http://www.mapcentia.com/dk/produkt/) som bruger [PostgreSQL](https://www.postgresql.org/), så derfor vil denne notebook være rettet mod at snakke med denne platform. 

### Om Geodata-info.dk
Geodata-info.dk er den danske geoportal, der gør det muligt for professionelle brugere samt borgere med interesse for geodata at søge efter geodatasæt og geodatatjenester. Geodata-info.dk omfatter desuden den danske søgetjeneste i henhold til INSPIRE-direktivet (Forordning nr. 976/2009 for så vidt angår nettjenesterne).

In [1]:
import requests
import xml.etree.ElementTree as ET

### Identifikator fra GDI

In [3]:
gid = 'fb9e06dc-9f02-42ac-bf1c-e0d662b81f4f'

In [4]:
url = f"https://geodata-info.dk/srv/api/records/{gid}/formatters/xml"
r = requests.request("GET", url)

### Beskrivelse af datasættet trækkes ud

In [5]:
root = ET.fromstring(r.text)

In [6]:
namespaces = {
    'gmd': 'http://www.isotc211.org/2005/gmd',
    'gco': 'http://www.isotc211.org/2005/gco'
}

In [7]:
path = "gmd:identificationInfo/gmd:MD_DataIdentification/gmd:abstract/gco:CharacterString"

In [8]:
abstact = root.find(path, namespaces)

In [9]:
abstact.text

'Registrering af naturtyper, som er beskyttet efter naturbeskyttelseslovens § 3.\n\nDisse er:\n·          søer og vandhuller med et areal på mindst 100 m2, \n·          moser, enge, strandenge, strandsumpe, heder og overdrev med et areal på mindst 2500 m2,\n·          "mosaikker" af ovennævnte naturtyper med et areal på mindst 2500 m2,\n·          visse udpegede vandløb  & \n·          alle moser i f. m. beskyttede vandhuller, søer eller vandløb.\nBeskyttelsen gælder uanset om der er en eller flere ejere. Tilstandsændring af beskyttet natur kræver tilladelse (dispensation).\nEt beskyttet areal kan i årenes løb ændre sig så meget, at det ikke længere er beskyttet. Omvendt kan et areal som i dag ikke er beskyttet, ændre sig, så det bliver det.\nDet er de faktiske forhold på arealet (størrelse, botanik, omlægningshyppighed m. m.) der afgør, om det er beskyttet eller ej. Registreringen er derfor vejledende\nHvorvidt et areal er beskyttet eller ej, vil i tvivlstilfælde blive afgjort ved en 